In [ ]:
import os
import sys
import shutil
import subprocess
sys.path.append("../")

from pathlib import Path

In [ ]:
from HandsFreeDocking.tools.Protein_Preparation import ProteinPreparation_Chimera

In [ ]:
resource_dir = Path("../examples/")
protein = resource_dir / "LAG3_Moloc_1.pdb"
crystal = resource_dir / "Fake_Crystal.sdf"
to_dock = resource_dir / "Fake_Crystal.sdf"

protein_abs = protein.resolve()
crystal_abs = crystal.resolve()
to_dock_abs = to_dock.resolve()

assert protein_abs.exists()
assert crystal_abs.exists()
assert to_dock_abs.exists()

In [ ]:
env = os.environ.copy()
env["RBT_ROOT"] = "/home/hitesit/Software/rxdock/rxdock_installation"
env["PATH"] = f"{env['RBT_ROOT']}/bin:{env.get('PATH', '')}"
env["LD_LIBRARY_PATH"] = f"{env['RBT_ROOT']}/lib/x86_64-linux-gnu:{env.get('LD_LIBRARY_PATH', '')}"
env["RBT_HOME"] = os.getcwd()

In [ ]:
# Run a command with subprocess
subprocess.run("rbdock", env=env)

In [ ]:
prep = ProteinPreparation_Chimera()
protein_mols = prep(protein_abs, Path("Protein.mol2"))

In [ ]:
prm = f"""RBT_PARAMETER_FILE_V1.00
TITLE gart_DUD
RECEPTOR_FILE {str(protein_mols)}
RECEPTOR_FLEX 3.0
##############################################
## CAVITY DEFINITION: REFERENCE LIGAND METHOD
##############################################
SECTION MAPPER
    SITE_MAPPER RbtLigandSiteMapper
    REF_MOL {str(crystal_abs)}
    RADIUS 6.0
    SMALL_SPHERE 1.0
    MIN_VOLUME 100
    MAX_CAVITIES 1
    VOL_INCR 0.0
    GRIDSTEP 0.5
END_SECTION
############################
## CAVITY RESTRAINT PENALTY
SECTION CAVITY
    SCORING_FUNCTION RbtCavityGridSF
    WEIGHT 1.0
END_SECTION
"""

prmtop_file = "probe.prmtop"
with open(prmtop_file, "w") as f:
    f.write(prm)

In [ ]:
params = [
    "rbcavity",
    "-W",
    "-d",
    "-r",
    "probe.prmtop"
]

result = subprocess.run(params, capture_output=True, text=True, env=env)
print(result.stdout)
if result.returncode != 0:
    print(f"Error: {result.stderr}")

In [ ]:
params = [
    "rbdock",
    "-i", str(to_dock),
    "-o", "docking_results",
    "-r", "probe.prmtop",
    "-p", "dock.prm",
    "-n", "50"
]

# Run the docking command
print(f"Running docking with command: {' '.join(params)}")
result = subprocess.run(params, capture_output=True, text=True, env=env)

# Print the output
print(result.stdout)